In [25]:
from bs4 import BeautifulSoup
import requests
import time

url = 'https://news.yahoo.co.jp/'
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")

list_item = soup.find_all('li')
#もしかしたらエラーの原因になるかも
list_item = list_item[12:21]

# <a>要素からトピックを取得
a_contents = [li.a for li in list_item]

articles_dict = []
topic = 'ライフ'
#トピックのURL取得
for i, a_content in enumerate(a_contents):
    if topic in a_content.text:
        categories_num = i

a_content = a_contents[categories_num]

topic_url = a_content['href']
URL = url + topic_url

r = requests.get(URL)
soup = BeautifulSoup(r.text, "html.parser")
#クラス名が変わる！！
elements = soup.select('.sc-dvXYtj.iduHXF')

elements = elements[:5]

news_urls = []
news_titles = []
#ニュースのURLとタイトルを取得
for element in elements:
    url = element.a['href']
    title = element.text[1:]
    news_urls.append(url)
    news_titles.append(title)
    time.sleep(0.2)

In [26]:
news_titles

['「暑すぎて終わった」猛暑でカップヌードル売れず？\u3000日清食品が吐いた本音にフォロー殺到ENCOUNT7/29(土)8:40',
 '「完全にやられた」孫の希望で迎えた豆柴に祖父が誰よりも“メロメロ”に…あまりの溺愛具合に家族が爆笑FNNプライムオンライン7/29(土)11:22',
 '木原副長官妻の取調官が異例の実名会見「事件はまだ終わっていない」不可解な捜査終結に激怒！日刊ゲンダイDIGITAL7/29(土)11:28',
 'テレビがなく、NHKは「ラジオ」でしか聞いていません。受信料の支払いは必要でしょうか？ファイナンシャルフィールド7/29(土)11:30',
 '添い寝で荒ぶる子ねこ\u3000必死の抵抗に悶絶する人が続出\u3000「怪獣化したちびたん」Hint-Pot7/29(土)8:10']

In [35]:
news_text = []
#ニュースの本文を取得
news_url = news_urls[0]
r = requests.get(news_url)
soup = BeautifulSoup(r.text, "html.parser")
#クラス名が変わるので注意！！                         
elements = soup.select('.sc-gDyJDg.dmrAcv.yjSlinkDirectlink.highLightSearchTarget')

In [36]:
elements

[<p class="sc-gDyJDg dmrAcv yjSlinkDirectlink highLightSearchTarget">　危険な暑さが続く中で、手が伸びやすいのは冷たいもの。そんな現状を大手食品メーカー「日清食品」が嘆くCMが注目されている。カップヌードルの公式X（ツイッター）アカウントは高温が続く現状に「夏暑すぎて終わったわ　カプヌうまいのに売れてない」と恨み節。思わぬ投稿に、商品のファンからは「いつ食べてもうまいよ！」というエールのほか、夏におススメのレシピが集まるなど大反響となっている。同社の宣伝部に聞いた。
 
 <a data-cl-params="_cl_vmodule:related;_cl_link:photo;_cl_position:1;view_pos:1;shcid:76574fcf3dd57e12d32c6e04e3b8b7d54d4b1101;url:https%3A%2F%2Fencount.press%2Farchives%2F492169%2F2%2F%3Futm_source%3Dyahoonews%26utm_medium%3Drss%26utm_campaign%3D492169_6;title:【動画】「夏暑すぎて終わったわ」　300万回以上再生されているカップヌードルの新CM;" href="https://encount.press/archives/492169/2/?utm_source=yahoonews&amp;utm_medium=rss&amp;utm_campaign=492169_6">【動画】「夏暑すぎて終わったわ」　300万回以上再生されているカップヌードルの新CM</a>
 
 　公式アカウントが「カプヌうまいのに売れてない」と投稿したのは7月18日のこと。画像や動画もない……、寂しい投稿にファンは仰天。1369万3000件も表示されたつぶやきには、「安心して下さい！食べてますよ！」「冷房でキンキンに冷えた部屋で食べるとうまい」「カプヌはいつ食べても美味いよ！！」というコメントと共に、9万5000件のいいねが集まり、担当者にエール。このほか、「夏はシーフードにライム搾ってナンプラー垂らしてパクチーをドサッ」「お湯少な目に入れて氷！！！！」「昔売ってた冷やして食べても美味しいそうめん

In [ ]:
    text = ""

    #ニュース記事の本文が配列で区切られていた場合
    for i in range(0, 8):
        try:
            if (i == 0):
                text = elements[0].text
            else:
                text += elements[i].text
        except:
            pass

    print(text)
    #ページが1つだけのとき、エラーが出るため
    try:
        #クラス名が変わるので注意！！ （取得例：２ページ)
        page_element = soup.select('.sc-ihRHuF.kFnlxQ')
        page = page_element[0].text         
        page_str = ''.join(filter(str.isdigit, page))
        page_num = int(page_str)
    except:
        page_num = 1
        
    #ページ数が複数のとき、ページ遷移し本文を取得
    if (page_num > 1):
        for i in range(2, page_num + 1):
            news_url_page = news_url + "?page=" + str(page_num)
            r = requests.get(news_url_page)
            soup = BeautifulSoup(r.text, "html.parser")
            #クラス名が変わるので注意！！ 
            elements = soup.select('.sc-gDyJDg.dmrAcv.yjSlinkDirectlink.highLightSearchTarget')
            #ニュース記事の本文が配列で区切られていた場合
            for i in range(0, 8):
                try:
                    text += elements[i].text
                except:
                    pass
                    
    news_text.append(text)

    time.sleep(1)

articles = []
for url, title, text in zip(news_urls, news_titles, news_text):
    article = {
        "title" : title,
        "URL" : url,
        "text" : text
    }
    articles.append(article)